In [1]:
with open('keywords') as fopen:
    keywords = fopen.read().split('\n')
keywords = [k for k in keywords if len(k)]
keywords = keywords * 200
len(keywords)

11800

In [2]:
len(keywords) * 20

236000

In [3]:
prompts = []
for k in keywords:
    prompt = f"""
```
camne nak buat RNN dalam tensorflow
camne nk buat websocket guna react
camne nk code simple landing page untuk kedai kasut
```

above are examples short coding question, generate 20 short coding questions related to {k}
"""
    prompts.append((k, prompt.strip()))

In [4]:
!rm -rf generate-questions
!mkdir generate-questions

In [5]:
import requests
import os
import re
import json

def answer(q, i):
    filename = os.path.join('generate-questions', f'{i}.json')
    
    if os.path.exists(filename) and os.path.getsize(filename) > 200:
        return
    
    for _ in range(5):
        try:
            headers = {
                'accept': 'application/json',
                'Content-Type': 'application/json',
            }
            
            messages = [
                {'role': 'user', 'content': q[1]}
            ]

            json_data = {
                'model': 'model',
                'messages': messages,
                'temperature': 1.0,
                'top_p': 0.8,
                'top_k': 50,
                'max_tokens': 1024,
                'repetition_penalty': 1.05,
            }

            response = requests.post('http://localhost:8000/v1/chat/completions', 
                                     headers=headers, json=json_data)
            r = response.json()
            r = r['choices'][0]['message']['content']
            if len(r.split()) > 20:
                with open(filename, 'w') as fopen:
                    json.dump({'q': q, 'r': r}, fopen)
            
                return
            else:
                print('condition not met', len(r.split()))
        except Exception as e:
            print(e)
            pass

In [6]:
def consumer(queue, name):
    while True:
        if queue.qsize() == 0:
            break
        item = queue.get()
        answer(*item)
    print(f'consumer {name} done')

In [7]:
from threading import Thread
from queue import Queue

queue = Queue()
urls = [(q, no) for no, q in enumerate(prompts)]
for u in urls:
    queue.put(u)
    
ori_size = queue.qsize()

In [8]:
answer(*urls[-2])

In [ ]:
from tqdm import tqdm

max_worker = 10
consumers = [Thread(target=consumer, args=(queue,i)) for i in range(max_worker)]
for i in range(len(consumers)):
    consumers[i].start()
    
pbar = tqdm(total=ori_size)
last_size = 0
while True:
    size = queue.qsize()
    if size == 0:
        break
    left = ori_size - size
    minus = left - last_size
    if minus > 0:
        pbar.update(minus)
        last_size += minus

pbar.close()

 90%|█████████████████████████████████████████████████████████████████████████▎       | 10676/11800 [3:11:07<13:39,  1.37it/s]